In [13]:
import pandas as pd

In [14]:
sales = pd.read_csv("../DataSet/start_vgsales.csv")
steam = pd.read_csv("../DataSet/start_steam-200k.csv", header=None).iloc[:,:-1]
steam.columns = ["player_id", "game_title", "type", "value"]

C:\Users\david\AppData\Local\Temp\ipykernel_11988\560633259.py:2: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  steam = pd.read_csv("../DataSet/start_steam-200k.csv", header=None).iloc[:,:-1]


In [15]:
steam = steam[steam["type"] == "purchase"]

In [16]:
df = steam.merge(sales, how='inner',left_on="game_title", right_on="Name")
df.head()

,player_id,game_title,type,value,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,151603712,Fallout 4,purchase,1.0,110,Fallout 4,PS4,2015.0,Role-Playing,Bethesda Softworks,2.47,3.15,0.24,1.10,6.96
1,151603712,Fallout 4,purchase,1.0,288,Fallout 4,XOne,2015.0,Role-Playing,Bethesda Softworks,2.45,1.26,0.01,0.37,4.09
2,151603712,Fallout 4,purchase,1.0,1632,Fallout 4,PC,2015.0,Role-Playing,Bethesda Softworks,0.50,0.63,0.00,0.10,1.23
3,87445402,Fallout 4,purchase,1.0,110,Fallout 4,PS4,2015.0,Role-Playing,Bethesda Softworks,2.47,3.15,0.24,1.10,6.96
4,87445402,Fallout 4,purchase,1.0,288,Fallout 4,XOne,2015.0,Role-Playing,Bethesda Softworks,2.45,1.26,0.01,0.37,4.09


In [17]:
df2 = df[["player_id", "Genre", "value"]]
df3 = df2.groupby(["player_id", "Genre"]).count()

In [18]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler

In [19]:
scaler = StandardScaler()
users = df3.reset_index()
users["player_id"] = np.asarray(users["player_id"]).astype(np.int32)
users[["value"]] = scaler.fit_transform(users[["value"]])
unique_user_ids = users["player_id"].unique()
users

,player_id,Genre,value
0,86540,Action,0.173283
1,86540,Adventure,-0.432836
2,86540,Racing,-0.069164
3,86540,Shooter,2.355311
4,86540,Strategy,-0.190388
...,...,...,...
8953,99723205,Shooter,0.779402
8954,99723205,Strategy,-0.554059
8955,99766416,Shooter,-0.432836
8956,99812428,Strategy,-0.432836


In [20]:
genres = users["Genre"].unique()
unique_genres = genres
genres

array(['Action', 'Adventure', 'Racing', 'Shooter', 'Strategy',
       'Role-Playing', 'Simulation', 'Sports', 'Fighting', 'Misc',
       'Puzzle', 'Platform'], dtype=object)

In [21]:
tf.random.set_seed(42)
users_tensor = tf.data.Dataset.from_tensor_slices(dict(users))
users_shuffled = users_tensor.shuffle(seed=42, buffer_size=100_000, reshuffle_each_iteration=False)

users_train = users_shuffled.take(8_000)
users_test = users_shuffled.skip(8_000).take(958)
print(len(users_test), users_test)

958 <TakeDataset element_spec={'player_id': TensorSpec(shape=(), dtype=tf.int32, name=None), 'Genre': TensorSpec(shape=(), dtype=tf.string, name=None), 'value': TensorSpec(shape=(), dtype=tf.float64, name=None)}>


In [22]:
genres_tensor = tf.data.Dataset.from_tensor_slices(genres)

In [23]:
for r in genres_tensor.take(5):
    print(r)

tf.Tensor(b'Action', shape=(), dtype=string)
tf.Tensor(b'Adventure', shape=(), dtype=string)
tf.Tensor(b'Racing', shape=(), dtype=string)
tf.Tensor(b'Shooter', shape=(), dtype=string)
tf.Tensor(b'Strategy', shape=(), dtype=string)


In [24]:
from typing import Dict, Text

In [32]:
class MultiTaskModel(tfrs.Model):
    # We derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.
  def __init__(
          self,
          user_model: tf.keras.Model,
          genre_model: tf.keras.Model,
          retrieval_task: tfrs.tasks.Retrieval,
          ranking_task: tfrs.tasks.Ranking,
          rating_weight:float = .5,
          retrieval_weight:float = .5,
  ):
    super().__init__()

      # Set up user and movie representations.
    self.user_model = user_model
    self.genre_model = genre_model

    # Compute predictions.
    self.rating_model = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
    ])

    
    self.rating_task: tf.keras.layers.Layer = ranking_task
    self.retrieval_task: tf.keras.layers.Layer = retrieval_task

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight


  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["player_id"])
    # And pick out the movie features and pass them into the movie model.
    genre_embeddings = self.genre_model(features["Genre"])

    return (
        user_embeddings,
        genre_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, genre_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("value")

    user_embeddings, genre_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, genre_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)




In [96]:
user_model = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(mask_token=None, vocabulary=unique_user_ids),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 64)
])
genre_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(mask_token=None, vocabulary=unique_genres),
    tf.keras.layers.Embedding(len(unique_genres) + 1, 64)
])

In [97]:
ranking_task = tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)

retrieval_task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=genres_tensor.batch(128).map(genre_model)
    )
)

In [125]:
model = MultiTaskModel(
    genre_model=genre_model, 
    user_model=user_model, 
    retrieval_task=retrieval_task, 
    ranking_task=ranking_task,
    rating_weight=.5,
    retrieval_weight=.5,
)
model.compile(optimizer=tf.keras.optimizers.Adagrad(.3))

In [126]:
cached_train = users_train.shuffle(100_000).batch(8192).cache()
cached_test = users_test.batch(4096).cache()

In [127]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_train, return_dict=True)

print(f"Retrieval top-5 accuracy: {metrics['factorized_top_k/top_5_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")


Epoch 1/30
1/1 [==============================] - 3s 3s/step - root_mean_squared_error: 1.5731 - factorized_top_k/top_1_categorical_accuracy: 0.1417 - factorized_top_k/top_5_categorical_accuracy: 0.4504 - factorized_top_k/top_10_categorical_accuracy: 0.8331 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 880992.3750 - regularization_loss: 0.0000e+00 - total_loss: 880992.3750
Epoch 2/30
1/1 [==============================] - 1s 827ms/step - root_mean_squared_error: 357.3353 - factorized_top_k/top_1_categorical_accuracy: 0.1587 - factorized_top_k/top_5_categorical_accuracy: 0.4846 - factorized_top_k/top_10_categorical_accuracy: 0.8577 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 652648.9375 - regularization_loss: 0.0000e+00 - total_loss: 652648.9375
Epoch 3/30
1/1 [==============================] - 1s 713ms/step - root_mean_squared_error: 407.73

In [128]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 141ms/step - root_mean_squared_error: 1.0360 - factorized_top_k/top_1_categorical_accuracy: 0.0104 - factorized_top_k/top_5_categorical_accuracy: 0.3413 - factorized_top_k/top_10_categorical_accuracy: 0.8058 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 18250.6211 - regularization_loss: 0.0000e+00 - total_loss: 18250.6211


{'root_mean_squared_error': 1.0359984636306763,
 'factorized_top_k/top_1_categorical_accuracy': 0.010438413359224796,
 'factorized_top_k/top_5_categorical_accuracy': 0.34133613109588623,
 'factorized_top_k/top_10_categorical_accuracy': 0.8058454990386963,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'loss': 18250.62109375,
 'regularization_loss': 0,
 'total_loss': 18250.62109375}

In [130]:
res = []
for genre in unique_genres:
  trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
        "player_id": np.array([97782908]),
        "Genre": np.array([genre])
    })
  res.append([genre, predicted_rating.numpy()[0][0]])
res = pd.DataFrame(data=res, columns = ["Genre", "Rating"])
res.sort_values(by="Rating", ascending=False)

,Genre,Rating
0,Action,0.012716
1,Adventure,0.012716
2,Racing,0.012716
3,Shooter,0.012716
4,Strategy,0.012716
5,Role-Playing,0.012716
6,Simulation,0.012716
7,Sports,0.012716
8,Fighting,0.012716
9,Misc,0.012716
